In [0]:
import os
from skimage.io import imread
from skimage.transform import resize
import matplotlib.pyplot as plt
import pickle
from IPython.display import clear_output

In [0]:
def try_to_load_as_pickled_object_or_None(filepath):
    """
    This is a defensive way to write pickle.load, allowing for very large files on all platforms
    """
    max_bytes = 2**31 - 1
    try:
        input_size = os.path.getsize(filepath)
        print(input_size)
        bytes_in = bytearray(0)
        with open(filepath, 'rb') as f_in:
            for _ in range(0, input_size, max_bytes):
                bytes_in += f_in.read(max_bytes)
        obj = pickle.loads(bytes_in)
    except:
        return None
    return obj

## Loading train and test features and labels

In [0]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1eY0XYoQTJe0sPsmIGfwTCH5faLdocaJ5' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1eY0XYoQTJe0sPsmIGfwTCH5faLdocaJ5" -O imgs_test.pkl && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1HTDlQ4g4-KAW7T6BjzEz8DQ4asFGzvVr' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1HTDlQ4g4-KAW7T6BjzEz8DQ4asFGzvVr" -O nums_test.pkl && rm -rf /tmp/cookies.txt
clear_output()

In [0]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1s5PSuD3RTtfUZTisSRC9Mhiq5c0mys3U' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1s5PSuD3RTtfUZTisSRC9Mhiq5c0mys3U" -O nums_train.pkl && rm -rf /tmp/cookies.txt
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1Nd5rVRjyIusrpa5BLiMtQVlxSQb9JFYv' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1Nd5rVRjyIusrpa5BLiMtQVlxSQb9JFYv" -O imgs_train.pkl && rm -rf /tmp/cookies.txt
clear_output()

In [0]:
features_train = try_to_load_as_pickled_object_or_None("hog_features_train.pkl")
features_test = try_to_load_as_pickled_object_or_None("hog_features_test.pkl")

nums_train = try_to_load_as_pickled_object_or_None("nums_train.pkl")
nums_test = try_to_load_as_pickled_object_or_None("nums_test.pkl")

124029522
31011282
41039
10267


In [0]:
!pip install catboost

     |████████████████████████████████| 62.9MB 120kB/s 


## Training Catboost
We have trained models on three hyperparameters setups:

1. iterations=500, learning_rate=0.9, depth=5
2. iterations=1000, learning_rate=0.9, depth=2
3. iterations=5000, learning_rate=0.5, depth=2

In [0]:
from catboost import Pool, CatBoostClassifier

cat_features = [0]

train_dataset = Pool(data=features_train,
                     label=nums_train)

eval_dataset = Pool(data=features_test,
                    label=nums_test)

# Initialize CatBoostClassifier
model = CatBoostClassifier(iterations=5000,
                           learning_rate=0.5,
                           depth=2,
                           task_type="GPU",
                           devices='0:1',
                           loss_function='MultiClass')
# Fit model
model.fit(train_dataset)

0:	learn: 4.0663943	total: 253ms	remaining: 21m 6s
1:	learn: 4.0440526	total: 489ms	remaining: 20m 21s
2:	learn: 4.0311348	total: 700ms	remaining: 19m 26s
3:	learn: 4.0093030	total: 862ms	remaining: 17m 56s
4:	learn: 3.9982933	total: 1.03s	remaining: 17m 8s
5:	learn: 3.9878734	total: 1.19s	remaining: 16m 27s
6:	learn: 3.9696567	total: 1.34s	remaining: 15m 56s
7:	learn: 3.9565817	total: 1.52s	remaining: 15m 48s
8:	learn: 3.9446887	total: 1.69s	remaining: 15m 35s
9:	learn: 3.9362143	total: 1.85s	remaining: 15m 24s
10:	learn: 3.9240415	total: 2.02s	remaining: 15m 14s
11:	learn: 3.9159135	total: 2.19s	remaining: 15m 12s
12:	learn: 3.9042190	total: 2.35s	remaining: 15m 1s
13:	learn: 3.8982454	total: 2.5s	remaining: 14m 52s
14:	learn: 3.8890940	total: 2.69s	remaining: 14m 52s
15:	learn: 3.8801934	total: 2.85s	remaining: 14m 46s
16:	learn: 3.8731157	total: 3s	remaining: 14m 39s
17:	learn: 3.8640634	total: 3.16s	remaining: 14m 35s
18:	learn: 3.8569912	total: 3.32s	remaining: 14m 29s
19:	learn:

## Testing CatBoost

In [0]:
# Get predicted classes
preds_class = model.predict(eval_dataset)

accuracy=0
for i in range(len(preds_class)):
  if preds_class[i]==nums_test[i]:
    accuracy+=1
accuracy = accuracy/len(nums_test)

print(accuracy*100)

9.410043881033642
